In [1]:
# ==============================================
# LOGIN + EXPORTACIÓN AUTOMÁTICA (WooCommerce)
# ==============================================
# Requisitos:
# pip install selenium pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
import time
from datetime import date, timedelta

# ==============================================
# 1) Leer datos del CSV
# ==============================================
df = pd.read_csv("Panetones.csv", dtype=str)
usuario = df.loc[0, "usuario"]
contrasena = df.loc[0, "contrasena"]

# ==============================================
# 2) Configurar Selenium
# ==============================================
url_login = "https://paneton.pucp.clinicasanjuandedioslima.com/wp-admin/"
url_export = "https://paneton.pucp.clinicasanjuandedioslima.com/wp-admin/admin.php?page=wc-order-export"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # Descomenta si no quieres que se abra la ventana del navegador

service = ChromeService()
driver = webdriver.Chrome(service=service, options=options)

# ==============================================
# 3) Login
# ==============================================
driver.get(url_login)
time.sleep(3)

# Usuario
driver.find_element(By.ID, "user_login").send_keys(usuario)

# Contraseña
driver.find_element(By.ID, "user_pass").send_keys(contrasena)

# Click en "Acceder"
driver.find_element(By.ID, "wp-submit").click()
time.sleep(5)

# ==============================================
# 4) Ir al módulo de exportación
# ==============================================
driver.get(url_export)
time.sleep(4)

# ==============================================
# 5) Rellenar los campos de fecha (día de ayer)
# ==============================================
# Calcula la fecha de ayer en formato YYYY-MM-DD
fecha = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
print(f"📅 Fecha usada para exportar: {fecha}")

# Campo "Desde"
campo_desde = driver.find_element(By.ID, "from_date")
campo_desde.clear()
campo_desde.send_keys(fecha)

# Campo "Hasta"
campo_hasta = driver.find_element(By.ID, "to_date")
campo_hasta.clear()
campo_hasta.send_keys(fecha)

time.sleep(1)

# ==============================================
# 6) Seleccionar formato XLS (radio button)
# ==============================================
radio_xls = driver.find_element(By.CSS_SELECTOR, "input.output_format[value='XLS']")
if not radio_xls.is_selected():
    radio_xls.click()

time.sleep(1)

# ==============================================
# 7) Hacer clic en "Exportar"
# ==============================================
btn_exportar = driver.find_element(By.ID, "export-btn")
btn_exportar.click()

time.sleep(5)  # Espera unos segundos a que termine la descarga

print("✅ Exportación ejecutada con éxito.")

# driver.quit()  # Descomenta si quieres cerrar el navegador automáticamente


📅 Fecha usada para exportar: 2025-11-04
✅ Exportación ejecutada con éxito.


In [14]:
# pip install pandas openpyxl unidecode
import pandas as pd
import glob, os, re, datetime
from unidecode import unidecode

# =========================
# 0) Carpeta Descargas
# =========================
base_dir = r"C:\Users\jcanalesa\Downloads"

# ---------------------------
# 1) Localizar el archivo más reciente orders-*.xlsx
# ---------------------------
pattern = os.path.join(base_dir, "orders-*.xlsx")
candidatos = glob.glob(pattern)
if not candidatos:
    raise FileNotFoundError(f"No se encontraron archivos con el patrón {pattern}")

archivo = max(candidatos, key=os.path.getmtime)
print(f"Usando archivo: {archivo}")

# ---------------------------
# 2) Leer Excel
# ---------------------------
df = pd.read_excel(archivo, dtype=str, engine="openpyxl").fillna("")

# ---------------------------
# 3) Normalizador
# ---------------------------
def norm(s: str) -> str:
    s = unidecode((s or "")).lower()
    s = re.sub(r"[()\[\]\-_/.,#:]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---------------------------
# 4) Encabezados objetivo
# ---------------------------
target_headers = [
    "Key","Estado del pedido","Fecha del pedido","Nota del cliente","Nombre facturacion",
    "Apellidos facturacion","Empresa facturacion","Direccion lineas 1 y 2 facturacion","Ciudad facturacion",
    "Correo electronico facturacion","Nombre envio","Apellidos envio","Direccion lineas 1 y 2 envio",
    "Ciudad envio","Codigo de provincia envio","Codigo postal envio","Codigo del pais envio",
    "Titulo del metodo de pago","Importe de descuento del carrito","Importe de subtotal del pedido",
    "DIRECCION DE ENVIO","Importe de envio del pedido","Importe reembolsado del pedido","Importe total del pedido",
    "Importe total de impuestos del pedido","DISTRITO","SKU","Articulo","Nombre del articulo","Cantidad",
    "PRECIO UNITARIO","Informacion paneton","Vinculo con PUCP","nro_documento","Telefono facturacion","Tipo",
    "Cantidad total de panetones","Condicion"
]
target_norm = {norm(c): c for c in target_headers}

aliases = {
    "telefono facturacion": "Telefono facturacion",
    "telefono (facturacion)": "Telefono facturacion",
    "telefono de facturacion": "Telefono facturacion",
    "nro documento": "nro_documento",
    "numero de documento": "nro_documento",
    "num documento": "nro_documento",
    "dni": "nro_documento",
    "documento": "nro_documento",
    "vinculo con pucp": "Vinculo con PUCP",
    "estado del pedido": "Estado del pedido",
    "direccion lineas 1 y 2 facturacion": "Direccion lineas 1 y 2 facturacion",
    "direccion lineas 1 y 2 envio": "Direccion lineas 1 y 2 envio",
}

# ---------------------------
# 5) Mapear nombres conocidos
# ---------------------------
rename_map = {}
for col in df.columns:
    n = norm(col)
    if n in target_norm:
        rename_map[col] = target_norm[n]
    elif n in aliases:
        rename_map[col] = aliases[n]
df = df.rename(columns=rename_map)

# ---------------------------
# 6) Detectar columna de Key
# ---------------------------
def guess_key_column(columns):
    patterns = [
        r"\bkey\b", r"\border\b", r"\border id\b", r"\border number\b",
        r"\bid\b", r"\bnumero\b", r"\bnro\b", r"\bpedido\b"
    ]
    for p in patterns:
        for c in columns:
            if re.search(p, norm(c)):
                return c
    return columns[0]

if "Key" not in df.columns or df["Key"].eq("").all():
    key_col = guess_key_column(list(df.columns))
    if key_col != "Key":
        df["Key"] = df[key_col].astype(str).fillna("")
        print(f"→ Columna usada como Key: '{key_col}'")

# ---------------------------
# 7) Reemplazo de valores
# ---------------------------
if "Estado del pedido" in df.columns:
    df["Estado del pedido"] = df["Estado del pedido"].replace({"Completado": "Sin Entregar"})
else:
    df["Estado del pedido"] = ""

# ---------------------------
# 7.1) Eliminar registros Fallido o Cancelado (case-insensitive)
# ---------------------------
if "Estado del pedido" in df.columns:
    before = len(df)
    estados_norm = df["Estado del pedido"].str.strip().str.lower()
    df = df[~estados_norm.isin(["fallido", "cancelado"])]
    removed = before - len(df)
    print(f"🧹 Registros eliminados por estado Fallido/Cancelado: {removed}")

# ---------------------------
# 8) Asegurar columnas faltantes
# ---------------------------
for col in target_headers:
    if col not in df.columns:
        df[col] = ""

# ---------------------------
# 9) Rellenar las columnas nuevas
# ---------------------------
df["Tipo"] = "E-commerce"
df["Condicion"] = "Activo"

def calcular_total(row):
    try:
        cantidad = float(row.get("Cantidad", 0))
    except ValueError:
        cantidad = 0
    nombre = row.get("Nombre del articulo", "").strip()
    if nombre == "Panetón solidario PUCP (caja x 6 unidades de 900gr)":
        return int(cantidad * 6)
    elif nombre == "Panetón solidario PUCP (caja 900gr)":
        return int(cantidad * 1)
    else:
        return ""
df["Cantidad total de panetones"] = df.apply(calcular_total, axis=1)

# ---------------------------
# 9.5) Formatear Fecha del pedido => d/mm/YYYY H:MM (sin ceros en d y H)
# ---------------------------
if "Fecha del pedido" in df.columns:
    s = df["Fecha del pedido"].astype(str).str.strip()

    # Crear Series vacía con el MISMO índice que s
    dt = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # 1) Seriales de Excel (e.g., 45678.45)
    mask_num = s.str.match(r"^\d+(\.\d+)?$", na=False)
    if mask_num.any():
        dt_num = pd.to_datetime(
            s[mask_num].astype(float),
            unit="d",
            origin="1899-12-30",
            errors="coerce"
        )
        dt.loc[mask_num] = dt_num

    # 2) Cadenas: primero asume m/d (dayfirst=False); luego d/m (dayfirst=True)
    mask_rest = dt.isna()
    if mask_rest.any():
        s_rest = s[mask_rest]
        parsed_mdy = pd.to_datetime(s_rest, errors="coerce", dayfirst=False)
        dt.loc[mask_rest] = parsed_mdy

        mask_still = dt.isna()
        if mask_still.any():
            s_still = s[mask_still]
            parsed_dmy = pd.to_datetime(s_still, errors="coerce", dayfirst=True)
            dt.loc[mask_still] = parsed_dmy

    # 3) Formato final (día y hora sin cero a la izquierda, según SO)
    if os.name == "nt":
        fmt = "%#d/%m/%Y %#H:%M"   # Windows
    else:
        fmt = "%-d/%m/%Y %-H:%M"   # Unix/Mac

    df["Fecha del pedido"] = dt.dt.strftime(fmt)
    # Reemplaza NaT formateado por vacío
    df["Fecha del pedido"] = df["Fecha del pedido"].fillna("").replace("NaT", "")

# ---------------------------
# 10) Reordenar columnas finales (principal)
# ---------------------------
df = df[target_headers]

# ---------------------------
# 11) Guardar resultado principal: sobrescribir Excel original + CSV
# ---------------------------
try:
    df.to_excel(archivo, index=False, engine="openpyxl")
    print(f"✅ Excel sobrescrito: {archivo}")
except PermissionError:
    raise PermissionError("No se pudo escribir el Excel. ¿Está abierto en otra aplicación? Ciérralo e intenta de nuevo.")

csv_path = os.path.splitext(archivo)[0] + ".csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ CSV generado: {csv_path}")

# ==============================================================
# 12) NUEVO: Generar Excel/CSV aparte con [Key, DIRECCION DE ENVIO(mapeada), Cantidad total de panetones]
# ==============================================================

def map_direccion_envio(val: str) -> str:
    """Mapea frases largas a 'Campus' o 'Delivery' (ignora acentos/caso/espacios)."""
    v = norm(val)
    campus_norm = norm("Entrega gratuita en el Campus PUCP: Almacén al lado de la Explanada del Polideportivo (disponible para miembros de la comunidad PUCP a partir de las 24 horas de compra).")
    delivery_norm = norm("Servicio delivery (Envío hasta 5 días hábiles)")
    if v == campus_norm or ("campus pucp" in v and "explanada del polideportivo" in v):
        return "Campus"
    if v == delivery_norm or "servicio delivery" in v or "delivery" in v:
        return "Delivery"
    # Si no coincide, devuelve el valor original
    return val

# Crear dataframe reducido
df_envio = pd.DataFrame({
    "Key": df["Key"],
    "DIRECCION DE ENVIO": df["DIRECCION DE ENVIO"].apply(map_direccion_envio),
    "Cantidad total de panetones": df["Cantidad total de panetones"]
})

# Rutas de salida (en la misma carpeta Downloads)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
out_xlsx = os.path.join(base_dir, f"orders-envio-resumen-{timestamp}.xlsx")
out_csv  = os.path.join(base_dir, f"orders-envio-resumen-{timestamp}.csv")

# Guardar
df_envio.to_excel(out_xlsx, index=False, engine="openpyxl")
print(f"✅ Excel de envío generado: {out_xlsx}")

df_envio.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"✅ CSV de envío generado: {out_csv}")



Usando archivo: C:\Users\jcanalesa\Downloads\orders-2025-11-05-14-25-44.xlsx
→ Columna usada como Key: 'Número de pedido'
🧹 Registros eliminados por estado Fallido/Cancelado: 2
✅ Excel sobrescrito: C:\Users\jcanalesa\Downloads\orders-2025-11-05-14-25-44.xlsx
✅ CSV generado: C:\Users\jcanalesa\Downloads\orders-2025-11-05-14-25-44.csv
✅ Excel de envío generado: C:\Users\jcanalesa\Downloads\orders-envio-resumen-20251105-142605.xlsx
✅ CSV de envío generado: C:\Users\jcanalesa\Downloads\orders-envio-resumen-20251105-142605.csv


In [3]:
# ==============================================
# LOGIN A KISSFLOW (usando el mismo driver activo) – PUCP rápido
# ==============================================
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# ---------- Datos ----------
url_kissflow = "https://pucp.kissflow.com/view/dataset/Base_de_datos_panetones_2"

df_kf = pd.read_csv("Kissflow.csv", dtype=str)
usuario = df_kf.loc[0, "usuario"]            # Correo institucional (ej. dgth.txd@pucp.edu.pe)
contrasena = df_kf.loc[0, "contrasena"]      # Misma contraseña para Google y PUCP
pucp = df_kf.loc[0, "pucp"]                  # Usuario PUCP (ej. jcanalesa)

wait = WebDriverWait(driver, 10)  # Esperas ágiles

def js_set_value(elem, value):
    driver.execute_script("""
        const el = arguments[0], val = arguments[1];
        el.focus(); el.value = val;
        el.dispatchEvent(new Event('input', {bubbles:true}));
        el.dispatchEvent(new Event('change', {bubbles:true}));
        el.dispatchEvent(new Event('blur', {bubbles:true}));
    """, elem, value)

def js_click(elem):
    driver.execute_script("arguments[0].click();", elem)

# ---------- Flujo principal ----------
driver.get(url_kissflow)

try:
    # 1️⃣ Botón "Continuar con Google"
    try:
        google_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class,'socialLoginItem')]//span[normalize-space()='Google']")))
        google_btn.click()
    except Exception:
        pass  # ya logueado quizá

    # 2️⃣ Correo Google (si aparece)
    try:
        email_input = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.ID, "identifierId")))
        email_input.clear()
        email_input.send_keys(usuario)
        driver.find_element(By.ID, "identifierNext").click()
    except Exception:
        pass

    # 3️⃣ Contraseña Google (si aparece)
    try:
        pwd_google = WebDriverWait(driver, 6).until(EC.visibility_of_element_located((By.NAME, "password")))
        pwd_google.clear()
        pwd_google.send_keys(contrasena)
        driver.find_element(By.ID, "passwordNext").click()
    except Exception:
        pass

    # 4️⃣ Botón "Continuar" (si aparece en flujo Google)
    try:
        btn_cont = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[normalize-space()='Continuar']]")))
        js_click(btn_cont)
    except Exception:
        pass

    # ==============================================================
    # 5️⃣ LOGIN SSO PUCP (optimizado y rápido)
    # ==============================================================

    # Si el login abrió nueva pestaña, cambiar a la última
    if len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[-1])

    # Esperar que aparezca el formulario (username/password)
    try:
        WebDriverWait(driver, 8).until(EC.any_of(
            EC.visibility_of_element_located((By.ID, "username")),
            EC.visibility_of_element_located((By.NAME, "username"))
        ))
    except TimeoutException:
        # Si no aparece el formulario, quizá ya estás autenticado
        pass

    # Completar usuario PUCP (rápido via JS)
    try:
        user_input = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.ID, "username")))
        user_input.clear()
        js_set_value(user_input, pucp)
    except Exception:
        pass

    # Completar contraseña PUCP (rápido via JS)
    try:
        pwd_input = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.ID, "password")))
        pwd_input.clear()
        js_set_value(pwd_input, contrasena)
    except Exception:
        pass

    # Click en ACCEDER (con fallback JS para evitar bloqueos)
    try:
        btn_acceder = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((
            By.XPATH,
            "//input[@type='submit' and translate(@value,'acder','ACDER')='ACCEDER'] | //button[contains(translate(.,'acder','ACDER'),'ACCEDER')]"
        )))
        try:
            btn_acceder.click()
        except (StaleElementReferenceException, Exception):
            js_click(btn_acceder)
    except TimeoutException:
        pass

    # 6️⃣ Segundo "Acceder" (ValidarNew) si aparece
    try:
        btn_acceder2 = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//input[@type='button' and contains(@onclick,'ValidarNew')]")))
        js_click(btn_acceder2)
    except Exception:
        pass

    # 7️⃣ Botón posterior (Continuar/Siguiente) tras ACCEDER
    try:
        siguiente = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((
            By.XPATH,
            "//button[.//span[normalize-space()='Continuar'] or .//span[normalize-space()='Siguiente'] or contains(translate(.,'siguiente','SIGUIENTE'),'SIGUIENTE')]"
        )))
        js_click(siguiente)
    except Exception:
        pass

    print("✅ Login a Kissflow completado.")

except Exception as e:
    print(f"❌ Error durante el login a Kissflow: {e}")

# driver.quit()  # Cierra el navegador si deseas


✅ Login a Kissflow completado.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time, os, glob

time.sleep(5)

# --- ya con sesión iniciada ---
# 1️⃣ Click en “Importar CSV”
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Importar CSV']]").click()
time.sleep(1)

# 2️⃣ Click en “Cargar”
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Cargar']]").click()
time.sleep(1)

# 3️⃣ Buscar el input tipo “file” dentro de la opción “Su dispositivo”
input_file = driver.find_element(By.XPATH, "//li[contains(@class,'fileupload')]//input[@type='file']")

# 4️⃣ Buscar el archivo automáticamente en Descargas
carpeta = r"C:\Users\jcanalesa\Downloads"
patron = os.path.join(carpeta, "orders-*.csv")
archivos = glob.glob(patron)

if archivos:
    csv_path = archivos[0]
    input_file.send_keys(csv_path)  # Subida directa, sin ventana
    print("✅ Archivo cargado en Kissflow:", csv_path)
else:
    print("❌ No se encontró el archivo CSV.")


time.sleep(5)
driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Siguiente']]").click()

driver.find_element(By.XPATH, "//button[.//span[normalize-space()='Siguiente']]").click()

# Pausa corta para asegurar carga (ajusta si es necesario)
time.sleep(2)

# 1️⃣ Seleccionar el radio button "whenImporting1"
driver.find_element(By.ID, "whenImporting1").click()
time.sleep(0.5)

# 2️⃣ Seleccionar el radio button "deleteDatasetRows1"
driver.find_element(By.ID, "deleteDatasetRows1").click()
time.sleep(0.5)

# 3️⃣ Seleccionar el radio button "inCaseOfErrors1"
driver.find_element(By.ID, "inCaseOfErrors1").click()
time.sleep(0.5)

✅ Archivo cargado en Kissflow: C:\Users\jcanalesa\Downloads\orders-2025-11-05-09-29-14.csv
